# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,la passe,45.5549,-0.8967,14.13,70,7,7.03,FR,1713373031
1,1,blackmans bay,-43.0167,147.3167,11.07,75,83,0.89,AU,1713373031
2,2,georgetown,5.4112,100.3354,28.97,74,20,1.54,MY,1713372900
3,3,lupane,-18.9315,27.8070,22.98,45,3,1.04,ZW,1713373031
4,4,bilibino,68.0546,166.4372,-23.20,97,57,1.89,RU,1713373032


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
map_plot_cities = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 900,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha =0.5
)
# Display the map
map_plot_cities

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
#city criteria: A max temperature lower than 27 degrees but higher than 21;Wind speed less than 4.5 m/s;Zero cloudiness
ideal_weather_df=city_data_df[(city_data_df["Max Temp"]>21)&(city_data_df["Max Temp"]<27)\
& (city_data_df["Wind Speed"]<4.5 )& (city_data_df["Cloudiness"]==0)]
# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()
# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
17,17,port elizabeth,-33.9180,25.5701,25.05,66,0,2.06,ZA,1713373035
102,102,formosa,-26.1775,-58.1781,25.54,47,0,0.89,AR,1713373055
246,246,esqueda,30.7167,-109.5833,22.02,13,0,0.93,MX,1713373089
250,250,murzuq,25.9155,13.9184,26.99,17,0,3.32,LY,1713373091
254,254,vail,32.0479,-110.7120,25.06,19,0,0.21,US,1713373091


### Step 3: Create a new DataFrame called `hotel_df`.

In [23]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
ideal_weather_copy=ideal_weather_df.copy()
hotel_df=ideal_weather_copy[["City","Country","Lat","Lng","Humidity"]]
hotel_df
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]= ""

# Display sample data
hotel_df.head()

/var/folders/v_/bwwf47r500n688q26f92ggbw0000gn/T/ipykernel_70156/3498645891.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"]= ""


,City,Country,Lat,Lng,Humidity,Hotel Name
17,port elizabeth,ZA,-33.9180,25.5701,66,
102,formosa,AR,-26.1775,-58.1781,47,
246,esqueda,MX,30.7167,-109.5833,13,
250,murzuq,LY,25.9155,13.9184,17,
254,vail,US,32.0479,-110.7120,19,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [26]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "limit":20,
    "apiKey":geoapify_key,
    "categories":"accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude=hotel_df.loc[index,"Lng"]
    latitude=hotel_df.loc[index,"Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
 
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()


Starting hotel search
port elizabeth - nearest hotel: Waterford Hotel
formosa - nearest hotel: Hotel Hostal del Rey
esqueda - nearest hotel: Hotel Esqueda
murzuq - nearest hotel: فندق باريس - مرزق
vail - nearest hotel: Hampton Inn & Suites Tucson Tech Park
laguna - nearest hotel: Holiday Inn Express & Suites
al artawiyah - nearest hotel: No hotel found
cafayate - nearest hotel: Il Cavallino
tazacorte - nearest hotel: App Leyma
east london - nearest hotel: No hotel found
port hedland - nearest hotel: The Esplanade Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
17,port elizabeth,ZA,-33.9180,25.5701,66,Waterford Hotel
102,formosa,AR,-26.1775,-58.1781,47,Hotel Hostal del Rey
246,esqueda,MX,30.7167,-109.5833,13,Hotel Esqueda
250,murzuq,LY,25.9155,13.9184,17,فندق باريس - مرزق
254,vail,US,32.0479,-110.7120,19,Hampton Inn & Suites Tucson Tech Park


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [15]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 1000,
    frame_height = 500,
    scale = 1,
    color = "City",
    alpha =0.5,
    hover_cols=["Country", "Hotel Name"]
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)